# Data

Loading and appending the Chicago Crime dataset

In [2]:
import pandas as pd
import duckdb
import openai
import time 
import os

Set the path of the CSV files:

In [37]:
path = "./data"

files = [x for x in os.listdir(path = path) if ".csv" in x]

print(files)

['Crimes_-_2021_20231227.csv', 'Crimes_-_2022_20231227.csv', 'Crimes_-_2023_20231227.csv']


In [138]:
chicago_crime = pd.concat((pd.read_csv(path +"/" + f) for f in files), ignore_index=True)
chicago_crime.columns = [c.strip().replace(" ", "_").lower() for c in chicago_crime.columns]
chicago_crime.head()

,id,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,...,ward,community_area,fbi_code,x_coordinate,y_coordinate,year,updated_on,latitude,longitude,location
0,25953,JE240540,05/24/2021 03:06:00 PM,020XX N LARAMIE AVE,0110,HOMICIDE,FIRST DEGREE MURDER,STREET,True,False,...,36.0,19,01A,1141387.0,1913179.0,2021,11/18/2023 03:39:49 PM,41.917838,-87.755969,"(41.917838056, -87.755968972)"
1,26038,JE279849,06/26/2021 09:24:00 AM,062XX N MC CORMICK RD,0110,HOMICIDE,FIRST DEGREE MURDER,PARKING LOT,True,False,...,50.0,13,01A,1152781.0,1941458.0,2021,11/18/2023 03:39:49 PM,41.995219,-87.713355,"(41.995219444, -87.713354912)"
2,12342615,JE202211,04/17/2021 03:20:00 PM,081XX S PRAIRIE AVE,0325,ROBBERY,VEHICULAR HIJACKING,RESIDENCE,True,False,...,6.0,44,03,1179448.0,1851073.0,2021,09/14/2023 03:41:59 PM,41.746626,-87.618032,"(41.746626309, -87.618031954)"
3,26262,JE366265,09/08/2021 04:45:00 PM,047XX W HARRISON ST,0110,HOMICIDE,FIRST DEGREE MURDER,CAR WASH,True,False,...,24.0,25,01A,1144907.0,1896933.0,2021,09/14/2023 03:41:59 PM,41.873191,-87.743447,"(41.873191445, -87.743446563)"
4,13209581,JG422927,08/01/2021 12:00:00 AM,012XX E 78TH ST,1563,SEX OFFENSE,CRIMINAL SEXUAL ABUSE,APARTMENT,False,False,...,8.0,45,17,NaN,NaN,2021,09/14/2023 03:43:09 PM,NaN,NaN,NaN


Source: https://platform.openai.com/examples/default-sql-translate

In [ ]:
Make sure to end the query with with a semicolon.
Only output the pure SQL code. Do not give an explanation for your answer.

In [139]:
prompt_template = """

Given the following SQL table, your job is to write queries given a user’s request, giving only code, no explanantion, and ending the query with a semicolon. \n

CREATE TABLE {} ({}) \n

Write a SQL query that returns - {}
"""

def sql_prompt_generator(table_name, col_names, query):
    prompt = prompt_template.format(table_name, col_names, query)
    return prompt


In [140]:
table = "chicago_crime"
col_names = str(list(chicago_crime.columns)).replace('[', '').replace(']', '')
query = "How many cases are from the year 2023?"

p = sql_prompt_generator(table_name = table, col_names = col_names, query = query)

print(p)





Given the following SQL table, your job is to write queries given a user’s request, giving only code, no explanantion, and ending the query with a semicolon. 


CREATE TABLE chicago_crime ('id', 'case_number', 'date', 'block', 'iucr', 'primary_type', 'description', 'location_description', 'arrest', 'domestic', 'beat', 'district', 'ward', 'community_area', 'fbi_code', 'x_coordinate', 'y_coordinate', 'year', 'updated_on', 'latitude', 'longitude', 'location') 


Write a SQL query that returns - How many cases are from the year 2023?



## Open AI API integration

In [3]:
from openai import OpenAI

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [35]:
openai_api_models = pd.DataFrame(client.models.list().data)
openai_api_models.sort_values(1, inplace=True, ascending=False)
openai_api_models.head()

,0,1,2,3
66,"(id, tts-1-hd-1106)","(created, 1699053533)","(object, model)","(owned_by, system)"
65,"(id, tts-1-1106)","(created, 1699053241)","(object, model)","(owned_by, system)"
46,"(id, tts-1-hd)","(created, 1699046015)","(object, model)","(owned_by, system)"
20,"(id, gpt-3.5-turbo-1106)","(created, 1698959748)","(object, model)","(owned_by, system)"
35,"(id, gpt-4-1106-preview)","(created, 1698957206)","(object, model)","(owned_by, system)"


In [141]:
def lang2sql(client, model, table, col_names, query):
    """
    Takes in a prompt and a query question and returns the corresponding dataframe
    """
    try:
        prompt = sql_prompt_generator(table_name = table, col_names = col_names, query = query)
        
        response = client.chat.completions.create(
            model=model,
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                }
            ],
        )

        response_text = response.choices[0].message.content
        startidtoken = "```sql"
        endidtoken = ";"
        sqlkey_select_loc = response_text.find(startidtoken)+len(startidtoken)
        sqlkey_semicolon_loc = response_text.find(endidtoken)
        query = response_text[sqlkey_select_loc:sqlkey_semicolon_loc]

        print("Executing query : ")
        print(query)

        query = query.replace("`", "\"")

        output_df = duckdb.sql(query).df()

        return prompt, response, output_df
    except Exception as e:
        print(e)
        return prompt, response, pd.DataFrame()


In [142]:
query = "How many cases are from the year 2023?"

prompt, response, output_df = lang2sql(client=client, model="gpt-4-1106-preview", table=table, col_names=col_names, query=query)

display(output_df)

Executing query : 

SELECT COUNT(*) AS total_cases_2023
FROM chicago_crime
WHERE year = 2023


,total_cases_2023
0,250979


In [143]:
query = "How many cases are from the year 2022?"

prompt, response, output_df = lang2sql(client=client, model="gpt-4-1106-preview", table=table, col_names=col_names, query=query)

display(output_df)

Executing query : 

SELECT COUNT(*) AS cases_in_2022
FROM chicago_crime
WHERE year = 2022


,cases_in_2022
0,239090


In [144]:
query = "how much did the number of cases increase from 2023 when compared with 2022?"

prompt, response, output_df = lang2sql(client=client, model="gpt-4-1106-preview", table=table, col_names=col_names, query=query)

display(output_df)

Executing query : 

SELECT 
    (SELECT COUNT(*) FROM chicago_crime WHERE year = 2023) - 
    (SELECT COUNT(*) FROM chicago_crime WHERE year = 2022) AS cases_increase


,cases_increase
0,11889


In [145]:
query = "How many cases ended up with arrest?"

prompt, response, output_df = lang2sql(client=client, model="gpt-4-1106-preview", table=table, col_names=col_names, query=query)

display(output_df)

Executing query : 

SELECT COUNT(*) AS arrest_count 
FROM chicago_crime 
WHERE arrest = 'TRUE'


,arrest_count
0,83666


In [149]:
# Table of chicago crime data from 2021-2023
chicago_crime.head(20)

,id,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,...,ward,community_area,fbi_code,x_coordinate,y_coordinate,year,updated_on,latitude,longitude,location
0,25953,JE240540,05/24/2021 03:06:00 PM,020XX N LARAMIE AVE,0110,HOMICIDE,FIRST DEGREE MURDER,STREET,True,False,...,36.0,19,01A,1141387.0,1913179.0,2021,11/18/2023 03:39:49 PM,41.917838,-87.755969,"(41.917838056, -87.755968972)"
1,26038,JE279849,06/26/2021 09:24:00 AM,062XX N MC CORMICK RD,0110,HOMICIDE,FIRST DEGREE MURDER,PARKING LOT,True,False,...,50.0,13,01A,1152781.0,1941458.0,2021,11/18/2023 03:39:49 PM,41.995219,-87.713355,"(41.995219444, -87.713354912)"
2,12342615,JE202211,04/17/2021 03:20:00 PM,081XX S PRAIRIE AVE,0325,ROBBERY,VEHICULAR HIJACKING,RESIDENCE,True,False,...,6.0,44,03,1179448.0,1851073.0,2021,09/14/2023 03:41:59 PM,41.746626,-87.618032,"(41.746626309, -87.618031954)"
3,26262,JE366265,09/08/2021 04:45:00 PM,047XX W HARRISON ST,0110,HOMICIDE,FIRST DEGREE MURDER,CAR WASH,True,False,...,24.0,25,01A,1144907.0,1896933.0,2021,09/14/2023 03:41:59 PM,41.873191,-87.743447,"(41.873191445, -87.743446563)"
4,13209581,JG422927,08/01/2021 12:00:00 AM,012XX E 78TH ST,1563,SEX OFFENSE,CRIMINAL SEXUAL ABUSE,APARTMENT,False,False,...,8.0,45,17,NaN,NaN,2021,09/14/2023 03:43:09 PM,NaN,NaN,NaN
5,13209369,JG422777,12/10/2021 12:00:00 AM,036XX S NORMAL AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,RESIDENCE,False,False,...,11.0,60,11,NaN,NaN,2021,09/14/2023 03:43:09 PM,NaN,NaN,NaN
6,12610088,JF134608,09/01/2021 12:01:00 AM,066XX S GREEN ST,0266,CRIMINAL SEXUAL ASSAULT,PREDATORY,RESIDENCE,True,False,...,16.0,68,02,NaN,NaN,2021,09/15/2023 03:41:25 PM,NaN,NaN,NaN
7,13211146,JG424812,01/01/2021 12:00:00 AM,035XX W DIVERSEY AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,APARTMENT,False,False,...,35.0,21,11,NaN,NaN,2021,09/15/2023 03:42:23 PM,NaN,NaN,NaN
8,13280741,JG507737,11/17/2021 10:40:00 AM,067XX S RIDGELAND AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,NaN,False,False,...,5.0,43,11,NaN,NaN,2021,11/18/2023 03:40:25 PM,NaN,NaN,NaN
9,12523398,JE423596,06/23/2021 12:00:00 AM,023XX W 24TH PL,1582,OFFENSE INVOLVING CHILDREN,CHILD PORNOGRAPHY,APARTMENT,False,False,...,25.0,31,17,NaN,NaN,2021,09/16/2023 03:42:58 PM,NaN,NaN,NaN


In [147]:
query = ""

prompt, response, output_df = lang2sql(client=client, model="gpt-4-1106-preview", table=table, col_names=col_names, query=query)

display(output_df)

In [148]:
# system = """
# Given the following SQL table, your job is to write queries given a user’s request. 


# CREATE TABLE chicago_crime (ID BIGINT, Case Number VARCHAR, Date VARCHAR, Block VARCHAR, IUCR VARCHAR, Primary Type VARCHAR, Description VARCHAR, Location Description VARCHAR, Arrest BOOLEAN, Domestic BOOLEAN, Beat BIGINT, District BIGINT, Ward DOUBLE, Community Area BIGINT, FBI Code VARCHAR, X Coordinate DOUBLE, Y Coordinate DOUBLE, Year BIGINT, Updated On VARCHAR, Latitude DOUBLE, Longitude DOUBLE, Location VARCHAR) 
# """

# prompt = "Write a SQL query that returns - How many cases ended up with arrest?"

# response = openai.ChatCompletion.create(
#   model="gpt-3.5-turbo",
#   messages=[
#     {
#       "role": "system",
#       "content": system
#     },
#     {
#       "role": "user",
#       "content": prompt
#     }
#   ],
#   temperature=1,
#   max_tokens=256,
#   top_p=1,
#   frequency_penalty=0,
#   presence_penalty=0
# )

In [37]:
print(response)

{
  "id": "chatcmpl-8PzRA3oj9bRqxq2xP45oL5PXVkvo6",
  "object": "chat.completion",
  "created": 1701205056,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "SELECT COUNT(*) FROM chicago_crime WHERE Arrest = TRUE;"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 134,
    "completion_tokens": 12,
    "total_tokens": 146
  }
}


In [14]:
  message = [
    {
      "role": "system",
      "content": prompt.system
    },
    {
      "role": "user",
      "content": prompt.user
    }
    ]

message

[{'role': 'system',
  'content': '\n\n    Given the following SQL table, your job is to write queries given a user’s request. \n\n\n    CREATE TABLE chicago_crime (ID BIGINT, Case Number VARCHAR, Date VARCHAR, Block VARCHAR, IUCR VARCHAR, Primary Type VARCHAR, Description VARCHAR, Location Description VARCHAR, Arrest BOOLEAN, Domestic BOOLEAN, Beat BIGINT, District BIGINT, Ward DOUBLE, Community Area BIGINT, FBI Code VARCHAR, X Coordinate DOUBLE, Y Coordinate DOUBLE, Year BIGINT, Updated On VARCHAR, Latitude DOUBLE, Longitude DOUBLE, Location VARCHAR) \n\n    '},
 {'role': 'user',
  'content': 'Write a SQL query that returns - How many cases ended up with arrest?'}]

In [15]:
 response = openai.ChatCompletion.create(
        model = "gpt-3.5-turbo",
        messages = message,
        temperature = 0,
        max_tokens = 256,
        frequency_penalty = 0,
        presence_penalty = 0)